In [1]:
import pandas as pd
import numpy as np
import pycountry
import zipcodes
from geopy.geocoders import Nominatim

In [2]:
# Read data
df_events = pd.read_csv('../Natural-Disasters.csv',delimiter=';')
df_location_data = pd.read_csv('../pend-gdis-1960-2018-disasterlocations.csv',delimiter=',')

C:\Users\jonas\AppData\Local\Temp\ipykernel_11788\1686743832.py:2: DtypeWarning: Columns (18,19,24,25,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv('../Natural-Disasters.csv',delimiter=';')


In [3]:
df_events

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"2,849084409",NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"2,849084409",NaN,NaN,NaN,NaN
2,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,843726.0,"2,963047785",NaN,NaN,NaN,NaN
3,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,"2,963047785",NaN,NaN,NaN,NaN
4,1902-0010-GTM,1902,10,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,"2,963047785",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,2023-0002-PHL,2023,2,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,12000.0,NaN,NaN,1;2,2357;2361;2362;2364;2368;67159;67162;67165;67166,24267;24272;67174,"Region II (Cagayan Valley), Region III (Centra..."
16541,2023-0056-ZMB,2023,56,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,65228;65231;65236,"Choma, Kalomo, Namwala (Adm2)."
16542,2023-0046-NZL,2023,46,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2171,NaN,Auckland (Adm1).
16543,2023-0032-USA,2023,32,NaN,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,3214;3224,NaN,"Alabama, Georgia (Adm1)."


In [4]:
# Drop all events from before 1960
df_events = df_events[df_events["Year"] >= 1960]

In [5]:
# Drop disasters with certain subgroups that aren't relevant to the scope
df_events = df_events[df_events["Disaster Subgroup"] != "Biological"]
df_events = df_events[df_events["Disaster Subgroup"] != "Extra-terrestrial"]

In [6]:
# Returns ISO-3 code for a given country name
def get_country_code(c_name):
    country = pycountry.countries.get(name=c_name)
    code = None
    if country is not None:
        code = country.alpha_3
    return code

In [7]:
# Fill in missing ISO3 codes
df_location_data.loc[df_location_data["iso3"].isna(), "iso3"] = df_location_data[df_location_data["iso3"].isna()].apply(lambda x: get_country_code(x["country"]),axis=1)

In [8]:
# Fill in missing ISO3 codes not included in the Pycountry library
df_location_data.loc[df_location_data["country"] == "Palestina", "iso3"] = "PSE"
df_location_data.loc[df_location_data["country"] == "Virgin Islands; U.S.", "iso3"] = "VIR"
df_location_data.loc[df_location_data["country"] == "Reunion", "iso3"] = "REU"
df_location_data.loc[df_location_data["country"] == "Saint Helena", "iso3"] = "SHN"
df_location_data.loc[df_location_data["country"] == "Micronesia", "iso3"] = "FSM"
df_location_data.loc[df_location_data["country"] == "British Virgin Islands", "iso3"] = "VGB"

In [9]:
# Edit disasterno to be the same as in df_events
df_location_data["disasterno"] = df_location_data["disasterno"] + "-" + df_location_data["iso3"]

In [10]:
def extract_lat_long(row, key):
    location_data = df_location_data[df_location_data["disasterno"] == row["Dis No"]]
    if not location_data.empty:
        return location_data.iloc[0, :][key]
    else:
        return None

In [11]:
# Augment Latitude info
df_events["Latitude"] = df_events.apply(lambda row: extract_lat_long(row, "latitude"), axis=1)

# Augment Longitude info
df_events["Longitude"] = df_events.apply(lambda row: extract_lat_long(row, "longitude"), axis=1)

In [12]:
df_events = df_events.drop(["Year", "Seq", "Glide", "Disaster Group", "Location", "OFDA Response", "Appeal", "Declaration", "Adm Level", "Admin1 Code", "Admin2 Code", "Geo Locations"], axis=1)

In [15]:
df_events

,Dis No,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,...,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI
303,1960-0013-CHL,Geophysical,Earthquake,Tsunami,NaN,NaN,Chile,CHL,South America,Americas,...,2000000.0,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,5442439.0,"10,10576299"
304,1960-0026-AIA,Meteorological,Storm,Tropical cyclone,NaN,Donna,Anguilla,AIA,Caribbean,Americas,...,NaN,1000.0,1250.0,NaN,NaN,NaN,NaN,35000.0,346337.0,"10,10576299"
305,1960-0025-ANT,Meteorological,Storm,Tropical cyclone,NaN,Donna,Netherlands Antilles,ANT,Caribbean,Americas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299"
306,1960-0024-ATG,Meteorological,Storm,Tropical cyclone,NaN,Donna,Antigua and Barbuda,ATG,Caribbean,Americas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299"
307,1960-0030-BGD,Meteorological,Storm,Tropical cyclone,NaN,NaN,Bangladesh,BGD,Southern Asia,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,2023-0002-PHL,Hydrological,Flood,NaN,NaN,NaN,Philippines (the),PHL,South-Eastern Asia,Asia,...,2000000.0,NaN,2000011.0,NaN,NaN,NaN,NaN,12000.0,NaN,NaN
16541,2023-0056-ZMB,Hydrological,Flood,NaN,NaN,NaN,Zambia,ZMB,Eastern Africa,Africa,...,22000.0,NaN,22000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16542,2023-0046-NZL,Hydrological,Flood,NaN,NaN,NaN,New Zealand,NZL,Australia and New Zealand,Oceania,...,3786.0,1614.0,5400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16543,2023-0032-USA,Meteorological,Storm,Convective storm,Tornado,NaN,United States of America (the),USA,Northern America,Americas,...,NaN,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [17]:
geolocator = Nominatim(user_agent='geoapiExercises')
def lat_long_to_state(lat,long):
    try:
        location = geolocator.reverse(f'{lat}, {long}')
    except:
        return None
    if (not location):
        return None
    address = location.raw['address']
    zip_code = address.get('postcode')
    state = address.get('state')
    if (state):
        return state
    if (not zip_code):
        return None
    
    if (len(zip_code) == 5 or len(zip_code) == 10):
        abrev = zipcodes.matching(f'{zip_code}')
        if (len(abrev) > 0):
            abrev = abrev[0]['state']
            return abbrev_to_us_state[abrev]
    return None

In [18]:
df_events['us state'] = df_events.apply(lambda row: lat_long_to_state(row['Latitude'],row['Longitude']), axis=1)

In [ ]:
df_events.to_csv('../Preprocessed-Natural-Disasters.csv', sep=";", index=False)